Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/using-mlflow/train-and-deploy-keras-auto-logging/train-and-deploy-keras-auto-logging.png)

## Use MLflow with Azure Machine Learning to Train and Deploy Keras Image Classifier

This example shows you how to use MLflow together with Azure Machine Learning services for tracking the metrics and artifacts while training a Keras model to classify MNIST digit images and deploy the model as a web service. You'll learn how to:

 1. Set up MLflow tracking URI so as to use Azure ML
 2. Create experiment
 3. Instrument your model with MLflow tracking
 4. Train a Keras model locally with MLflow auto logging
 5. Train a model on GPU compute on Azure with MLflow auto logging
 6. View your experiment within your Azure ML Workspace in Azure Portal
 7. Deploy the model as a web service on Azure Container Instance
 8. Call the model to make predictions
 
### Pre-requisites
 
If you are using a Notebook VM, you are all set. Otherwise, go through the [Configuration](../../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

Install TensorFlow and Keras, this notebook has been tested with TensorFlow version 2.1.0 and Keras version 2.3.1.

Also, install azureml-mlflow package using ```pip install azureml-mlflow```. Note that azureml-mlflow installs mlflow package itself as a dependency if you haven't done so previously.

### Set-up

Import packages and check versions of Azure ML SDK and MLflow installed on your computer. Then connect to your Workspace.

In [1]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace


print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)

SDK version: 1.42.0
MLflow version: 1.26.1


In [2]:
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourceGroups/mlservices/providers/Microsoft.MachineLearningServices/workspaces/msmls',
 'name': 'msmls',
 'identity': {'principal_id': '533f9b41-fe01-444e-be02-dac3fb70c11c',
  'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'type': 'SystemAssigned'},
 'location': 'australiaeast',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'c96ff9a0-71e5-4401-921d-660ad6b6335a',
 'sdkTelemetryAppInsightsKey': 'bd22fe07-ca7e-4337-bd98-3d5116261c1f',
 'description': '',
 'friendlyName': '',
 'containerRegistry': '/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/mlservices/providers/microsoft.containerregistry/registries/msmls5213556472',
 'keyVault': '/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/mlservices/providers/microsoft.keyvault/vaults/msmls5567188872',
 'applicationInsights': '/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/res

### Set tracking URI

Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [3]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Create Experiment

In both MLflow and Azure ML, training runs are grouped into experiments. Let's create one for our experimentation.

In [4]:
experiment_name = "keras-with-mlflow"
mlflow.set_experiment(experiment_name)

2022/06/06 23:09:56 INFO mlflow.tracking.fluent: Experiment with name 'keras-with-mlflow' does not exist. Creating a new experiment.


<Experiment: artifact_location='', experiment_id='f6578cde-4e14-4735-bde7-b955b5d16e1f', lifecycle_stage='active', name='keras-with-mlflow', tags={}>

### Train model locally while logging metrics and artifacts

The ```scripts/train.py``` program contains the code to load the image dataset, train and test the model. Within this program, the train.driver function wraps the end-to-end workflow.

Within the driver, the ```mlflow.start_run``` starts MLflow tracking. Then, MLflow's automatic logging is used to log metrics, parameters and model for the Keras run.

Let's add the program to search path, import it as a module and invoke the driver function. Note that the training can take few minutes.

In [5]:
lib_path = os.path.abspath("scripts")
sys.path.append(lib_path)

import train

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
run = train.driver()

### Train model on GPU compute on Azure

Next, let's run the same script on GPU-enabled compute for faster training. If you've completed the the [Configuration](../../../configuration.ipnyb) notebook, you should have a GPU cluster named "gpu-cluster" available in your workspace. Otherwise, follow the instructions in the notebook to create one. For simplicity, this example uses single process on single VM to train the model.

Clone an environment object from the Tensorflow 2.1 Azure ML curated environment. Azure ML curated environments are pre-configured environments to simplify ML setup, reference [this doc](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-a-curated-environment) for more information. To enable MLflow tracking, add ```azureml-mlflow``` as pip package.

In [ ]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="AzureML-TensorFlow-2.1-GPU").clone("mlflow-env")

env.python.conda_dependencies.add_pip_package("azureml-mlflow")
env.python.conda_dependencies.add_pip_package("keras==2.3.1")
env.python.conda_dependencies.add_pip_package("numpy")

Create a ScriptRunConfig to specify the training configuration: script, compute as well as environment.

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="./scripts", script="train.py")
src.run_config.environment = env
src.run_config.target = "gpu-cluster"

Get a reference to the experiment you created previously, but this time, as an Azure Machine Learning experiment object.

Then, use the ```Experiment.submit``` method to start the remote training run. Note that the first training run often takes longer as Azure Machine Learning service builds the Docker image for executing the script. Subsequent runs will be faster as the cached image is used.

In [ ]:
from azureml.core import Experiment

exp = Experiment(ws, experiment_name)
run = exp.submit(src)

You can monitor the run and its metrics on Azure Portal.

In [ ]:
run

Also, you can wait for run to complete.

In [ ]:
run.wait_for_completion(show_output=True)

### Deploy model as web service

The ```client.create_deployment``` function registers the logged Keras+Tensorflow model and deploys the model in a framework-aware manner. It automatically creates the Tensorflow-specific inferencing wrapper code and specifies package dependencies for you. See [this doc](https://mlflow.org/docs/latest/models.html#id34) for more information on deploying models on Azure ML using MLflow.

In this example, we deploy the Docker image to Azure Container Instance: a serverless compute capable of running a single container. You can tag and add descriptions to help keep track of your web service. 

[Other inferencing compute choices](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where) include Azure Kubernetes Service which provides scalable endpoint suitable for production use.

Note that the service deployment can take several minutes.

First define your deployment target and customize parameters in the deployment config. Refer to [this documentation](https://docs.microsoft.com/azure/machine-learning/reference-azure-machine-learning-cli#azure-container-instance-deployment-configuration-schema) for more information. 

In [ ]:
import json
  
# Data to be written
deploy_config ={
    "computeType": "aci"
}
# Serializing json 
json_object = json.dumps(deploy_config)
  
# Writing to sample.json
with open("deployment_config.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
from mlflow.deployments import get_deploy_client

# set the tracking uri as the deployment client
client = get_deploy_client(mlflow.get_tracking_uri())

# set the model path 
model_path = "model"

# set the deployment config
deployment_config_path = "deployment_config.json"
test_config = {'deploy-config-file': deployment_config_path}

# define the model path and the name is the service name
# the model gets registered automatically and a name is autogenerated using the "name" parameter below 
client.create_deployment(model_uri='runs:/{}/{}'.format(run.id, model_path),
                         config=test_config,
                         name="keras-aci-deployment")

Once the deployment has completed you can check the scoring URI of the web service in AzureML studio UI in the endpoints tab. Refer [mlflow predict](https://mlflow.org/docs/latest/python_api/mlflow.deployments.html#mlflow.deployments.BaseDeploymentClient.predict) on how to test your deployment. 

## Clean up
You can delete the ACI deployment with a delete API call.

In [ ]:
client.delete("keras-aci-deployment")